# Backpropagation

## 文件结构

```
backpropagation
├── dataset
│   ├── iris.txt
│   └── cancer.txt
├── parameters
│   ├── iris_hidden_parameters.txt
│   ├── iris_output_parameters.txt
│   ├── cancer_hidden_parameters.txt
│   └── cancer_output_parameters.txt
├── back_propagation.ipynb
└── back_propagation.py
```

## 符号约定

- `m`: sample number
- `d`: input feature
- `q`: hidden feature
- `l`: output feature
- `X`: samples, shape(m, d)
- `Y`: labels, shape(m, l)
- `hidden_weights`: weights of hidden layer nodes, shape(q, d)
- `hidden_bias`: bias of hidden layer nodes, shape(q, 1)
- `output_weights`: weights of output layer nodes, shape(l, q)
- `output_bias`: bias of output layer nodes, shape(l, 1)

## Backpropagation 算法及模型介绍

Backpropagation 算法（BP 算法）用于在多层神经网络中求解参数的梯度，并将梯度用于梯度下降算法，从而优化模型的损失函数。本实验中使用双层神经网络模型（一层隐藏层，一层输出层），激活函数为 Sigmoid 函数 $$\operatorname{Sigmoid}\left(z\right)=\frac{1}{1 + \exp\left(-z\right)}，$$ 损失函数为均方误差 $$E_k=\frac{1}{2}\sum\limits_{j=1}^{l}{\left(\hat{y}_j^k-y_j^k\right)^2}。$$ 本实验使用鸢尾花数据集和癌症数据集对模型进行测试。

鸢尾花数据集包含 150 个样本，本实验中将其随机分成 120 个训练样本和 30 个测试样本。每个样本有 4 个特征。有 3 个分类标签（编号为 0，1，2）。对鸢尾花数据集进行分类时选取隐藏层节点数 `q` 为 12。鸢尾花数据集的

癌症数据集包含 569 个样本，本实验中将其随机分为 450 个训练样本和 119 个测试样本。每个样本有 30 个特征。有 2 个分类标签（编号为 0，1）。对癌症数据集进行分类时选取隐藏层节点数 `q` 为 200。

# 引入相关的库

In [1]:
import numpy as np
import os

# 加载数据集及数据预处理

In [2]:
def normalization(
    X: np.matrix
) -> np.matrix:
    """
    将输入数据标准化。
    input:
        X: 输入数据 shape(m, d)
    output:
        标准化数据 shape(m, d)
    """
    m = X.shape[0]
    d = X.shape[1]
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return np.matrix(data=[[
        (X[k, i] - mean[0, i]) / std[0, i]
        for i in range(d)]
        for k in range(m)],
        dtype=np.float64)


def make_one_hot(
    Y: np.matrix
) -> np.matrix:
    """
    将标签值转化为独热码。
    input:
        Y: 标签值 shape(m, 1)
    output:
        one-hot: 独热码 shape(m, l)
    """
    return np.matrix(data=[
        np.eye(1, Y.max() + 1, y[0, 0], dtype=np.float64)[0, :]
        for y in Y],
        dtype=np.float64)


def load_data(
    file: str,
    training_number: int,
    sep: str = ','
) -> tuple:
    """
    从文件中加载数据。要求数据文件每行包含一个样例且每行最后一个数据为样例标签。
    input:
        file: 数据文件路径。
        training_number: 训练样本数。
        sep: 文件中每个特征的分隔符。
    output:
        X_training: 训练样本。
        Y_training: 训练样本标签。
        X_test: 测试样本。
        Y_test: 测试样本标签。
    """
    with open(file=file, mode='r') as f:
        data = np.matrix(data=[[
            np.float64(feature)
            for feature in sample.strip().split(sep=sep)]
            for sample in f.readlines()],
            dtype=np.float64)

    X = data[:, :-1]
    Y = np.matrix(data=[[int(y)] for y in data[:, -1]], dtype=np.int32)

    X = normalization(X=X)
    Y = make_one_hot(Y=Y)

    X_training = X[:training_number, :]
    Y_training = Y[:training_number, :]
    X_test = X[training_number:, :]
    Y_test = Y[training_number:, :]

    return X_training, Y_training, X_test, Y_test

## 鸢尾花数据集

In [3]:
iris_X_training, iris_Y_training, iris_X_test, iris_Y_test = load_data(
    file=os.path.join('dataset', 'iris.txt'),
    training_number=120)

## 癌症数据集

In [4]:
cancer_X_training, cancer_Y_training, cancer_X_test, cancer_Y_test = load_data(
    file=os.path.join('dataset', 'cancer.txt'),
    training_number=450,
    sep='\t')

# 加载参数
可以从文件中加载参数，也可以随机初始化参数。
本实验中在鸢尾花数据集和癌症数据集中分别采用两种方法。

In [5]:
def load_layer(
    file: str,
    sep: str = ','
) -> tuple:
    """
    从文件中加载某一层网络的参数。
    input:
        file: 保存参数的文件路径。
        sep: 参数之间的分隔符。
    output:
        weights: 权重。
        bias: 阈值。
    """
    with open(file=file, mode='r') as f:
        parameters = np.matrix(data=[[
            np.float64(parameter)
            for parameter in classifier.strip().split(sep=sep)]
            for classifier in f.readlines()])
    weights = parameters[:, :-1]
    bias = parameters[:, -1]

    return weights, bias

## 加载鸢尾花数据集的参数

In [6]:
iris_hidden_weights, iris_hidden_bias = load_layer(
    file=os.path.join('parameters', 'iris_hidden_parameters.txt'))
iris_output_weights, iris_output_bias = load_layer(
    file=os.path.join('parameters', 'iris_output_parameters.txt'))

## 加载癌症数据集的参数

In [7]:
cancer_hidden_weights, cancer_hidden_bias = np.random.rand(
    200, 30), np.random.rand(200, 1)
cancer_output_weights, cancer_output_bias = np.random.rand(
    2, 200), np.random.rand(2, 1)

# BP 算法实现

In [8]:
def sigmoid(
    z: np.matrix
) -> np.matrix:
    """
    Sigmoid(z) = 1 / (1 + exp(-z))。
    """
    return np.float64(1.) / (np.float64(1.) + np.exp(-z))


def back_propagation(
    X: np.matrix,
    Y: np.matrix,
    hidden_weights: np.matrix,
    hidden_bias: np.matrix,
    output_weights: np.matrix,
    output_bias: np.matrix,
    lr: np.float64
) -> tuple:
    """
    反向传播更新参数。
    input:
        X: 输入样本 shape(m, d)
        Y: 样本标签 shape(m, l)
        hidden_weights: 隐藏层权重 shape(q, d)
        hidden_bias: 隐藏层阈值 shape(q, 1)
        output_weights: 输出层权重 shape(l, q)
        output_bias: 输出层阈值 shape(l, 1)
        lr: 学习率
    """
    m = X.shape[0]
    d = X.shape[1]
    q = hidden_weights.shape[0]
    l = output_weights.shape[0]

    alpha = np.zeros(shape=(q, 1), dtype=np.float64)
    b = np.zeros(shape=(q, 1), dtype=np.float64)
    beta = np.zeros(shape=(l, 1), dtype=np.float64)
    y_hat = np.zeros(shape=(l, 1), dtype=np.float64)

    g = np.zeros(shape=(l, 1), dtype=np.float64)
    e = np.zeros(shape=(q, 1), dtype=np.float64)

    d_w = np.zeros(shape=(l, q), dtype=np.float64)
    d_theta = np.zeros(shape=(l, 1), dtype=np.float64)
    d_v = np.zeros(shape=(q, d), dtype=np.float64)
    d_gamma = np.zeros(shape=(q, 1), dtype=np.float64)

    for k in range(m):
        for h in range(q):
            sum_tmp = 0
            for i in range(d):
                sum_tmp += hidden_weights[h, i] * X[k, i]
            alpha[h, 0] = sum_tmp

        b = sigmoid(alpha - hidden_bias)

        for j in range(l):
            sum_tmp = 0
            for h in range(q):
                sum_tmp += output_weights[j, h] * b[h, 0]
            beta[j, 0] = sum_tmp

        y_hat = sigmoid(beta - output_bias)

        for j in range(l):
            g[j] = y_hat[j, 0] * (1 - y_hat[j, 0]) * (Y[k, j] - y_hat[j, 0])

        for h in range(q):
            sum_tmp = 0
            for j in range(l):
                sum_tmp += output_weights[j, h] * g[j, 0]
            e[h] = b[h, 0] * (1 - b[h, 0]) * sum_tmp

        for h in range(q):
            for j in range(l):
                d_w[j, h] = lr * g[j, 0] * b[h, 0]

        for j in range(l):
            d_theta[j, 0] = -lr * g[j, 0]

        for i in range(d):
            for h in range(q):
                d_v[h, i] = lr * e[h, 0] * X[k, i]

        for h in range(q):
            d_gamma[h, 0] = -lr * e[h, 0]

        output_weights += d_w
        output_bias += d_theta
        hidden_weights += d_v
        hidden_bias += d_gamma

# 训练模型
本步骤耗时较长（约 1 分 20 秒），请耐心等待。

In [9]:
epoch = 100
for i in range(epoch):
    back_propagation(
        X=iris_X_training,
        Y=iris_Y_training,
        hidden_weights=iris_hidden_weights,
        hidden_bias=iris_hidden_bias,
        output_weights=iris_output_weights,
        output_bias=iris_output_bias,
        lr=0.05)

epoch = 10
for i in range(epoch):
    back_propagation(
        X=cancer_X_training,
        Y=cancer_Y_training,
        hidden_weights=cancer_hidden_weights,
        hidden_bias=cancer_hidden_bias,
        output_weights=cancer_output_weights,
        output_bias=cancer_output_bias,
        lr=0.01)

# 预测结果

In [10]:
def predict(
    X: np.matrix,
    hidden_weights: np.matrix,
    hidden_bias: np.matrix,
    output_weights: np.matrix,
    output_bias: np.matrix
) -> np.matrix:
    """
    根据权重和样本特征预测标签。
    input:
        X: 输入样本 shape(m, d)
        hidden_weights: 隐藏层权重 shape(q, d)
        hidden_bias: 隐藏层阈值 shape(q, 1)
        output_weights: 输出层权重 shape(l, q)
        output_bias: 输出层阈值 shape(l, 1)
    output:
        prediction: 标签的预测值 shape(m, l)
    """
    m = X.shape[0]
    d = X.shape[1]
    q = hidden_weights.shape[0]
    l = output_weights.shape[0]

    alpha = np.zeros(shape=(q, 1), dtype=np.float64)
    b = np.zeros(shape=(q, 1), dtype=np.float64)
    beta = np.zeros(shape=(l, 1), dtype=np.float64)
    y_hat = np.zeros(shape=(l, 1), dtype=np.float64)

    prediction = np.zeros(shape=(m, l), dtype=np.float64)

    for k in range(m):
        for h in range(q):
            sum_tmp = 0
            for i in range(d):
                sum_tmp += hidden_weights[h, i] * X[k, i]
            alpha[h, 0] = sum_tmp

        b = sigmoid(alpha - hidden_bias)

        for j in range(l):
            sum_tmp = 0
            for h in range(q):
                sum_tmp += output_weights[j, h] * b[h, 0]
            beta[j, 0] = sum_tmp

        y_hat = sigmoid(beta - output_bias)

        for j in range(l):
            prediction[k, j] = y_hat[j, 0]

    return prediction

## 鸢尾花预测

In [11]:
iris_prediction = predict(
    X=iris_X_test,
    hidden_weights=iris_hidden_weights,
    hidden_bias=iris_hidden_bias,
    output_weights=iris_output_weights,
    output_bias=iris_output_bias)

print('iris prediction:\n{}\n'.format(np.argmax(iris_prediction, axis=1)))
print('iris Y test\n{}\n'.format(np.argmax(iris_Y_test, axis=1).T))

iris prediction:
[2 0 2 1 1 0 1 0 0 2 0 1 2 1 1 1 0 1 2 1 0 0 0 1 0 2 1 0 2 0]

iris Y test
[[2 0 2 1 1 0 1 0 0 2 0 1 2 1 1 1 0 1 2 1 0 0 0 1 0 2 1 0 2 0]]



## 癌症预测

In [12]:
cancer_prediction = predict(
    X=cancer_X_test,
    hidden_weights=cancer_hidden_weights,
    hidden_bias=cancer_hidden_bias,
    output_weights=cancer_output_weights,
    output_bias=cancer_output_bias)

print('cancer prediction:\n{}\n'.format(np.argmax(cancer_prediction, axis=1)))
print('cancer Y test:\n{}\n'.format(np.argmax(cancer_Y_test, axis=1).T))

cancer prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0
 0 1 1 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 0
 1 1 1 1 1 1 1 1]

cancer Y test:
[[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1
  1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1
  0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1
  1 1 1 1 1 1 1 1 1 1 1]]



# 测试模型

In [13]:
def MSE(
    prediction: np.matrix,
    Y: np.matrix
) -> np.float64:
    """
    计算预测值与真实值的均方误差。
    input:
        prediction: 预测值 shape(m, l)
        Y: 真实值 shape(m, l)
    output:
        均方误差值。
    """
    m = Y.shape[0]
    l = Y.shape[1]
    return np.mean(np.matrix(data=[0.5 * np.sum([
        (prediction[k, j] - Y[k, j]) ** 2
        for j in range(l)])
        for k in range(m)], dtype=np.float64))


def test(
    prediction: np.matrix,
    Y: np.matrix
) -> None:
    """
    根据给定的测试集和模型参数、测试模型的正确率。
    input:
        prediction: 标签的预测值 shape(1, m)
        Y: 标签的实际值 shape(m, 1)
    """
    m = prediction.shape[0]
    print("MSE: {}".format(
        MSE(prediction=prediction, Y=Y)))

    right = 0
    error = 0
    for k in range(m):
        if np.argmax(prediction[k, :]) == np.argmax(Y[k, :]):
            right += 1
        else:
            error += 1
    print("Right: {}, error: {}, right rate: {}".format(
        right, error, 1 if m == 0 else right / (right + error)))

## 鸢尾花测试

In [14]:
test(
    prediction=iris_prediction,
    Y=iris_Y_test)

MSE: 0.008520959533903544
Right: 30, error: 0, right rate: 1.0


## 癌症测试

In [15]:
test(
    prediction=cancer_prediction,
    Y=cancer_Y_test)

MSE: 0.2651738723428339
Right: 107, error: 12, right rate: 0.8991596638655462


# 保存参数

In [16]:
def save_layer(
    file: str,
    weights: np.matrix,
    bias: np.matrix,
    sep: str = ','
) -> None:
    """
    保存某一层网络的参数。
    input:
        file: 保存参数的文件路径。
        weights: 权重。
        bias: 阈值。
        sep: 参数之间的分隔符。
    """
    with open(file=file, mode='w') as f:
        line, row = weights.shape
        for i in range(line):
            for j in range(row):
                f.write('{}'.format(weights[i, j]) + sep)
            f.write('{}\n'.format(bias[i, 0]))

## 保存鸢尾花模型参数

In [17]:
save_layer(
    file=os.path.join('parameters', 'iris_hidden_parameters.txt'),
    weights=iris_hidden_weights,
    bias=iris_hidden_bias)

save_layer(
    file=os.path.join('parameters', 'iris_output_parameters.txt'),
    weights=iris_output_weights,
    bias=iris_output_bias)

## 保存癌症数据集参数

In [18]:
save_layer(
    file=os.path.join('parameters', 'cancer_hidden_parameters.txt'),
    weights=cancer_hidden_weights,
    bias=cancer_hidden_bias)

save_layer(
    file=os.path.join('parameters', 'cancer_output_parameters.txt'),
    weights=cancer_output_weights,
    bias=cancer_output_bias)